<a href="https://colab.research.google.com/github/josebel78/03MIAR_Algoritmos-de-Optimizacion/blob/main/Algoritmos_Jose_Belenguer_AG3_reto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TFM_VIU_09MIAR_JBB_Test
## José Belenguer Ballester
### Enlace del repositorio GitHub:
#### https://github.com/josebel78/TFM_VIU_09MIAR_JBB.git

## MODULE IMPORTS

In [1]:
import copy
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import random
import time
import warnings

In [2]:
warnings.filterwarnings("ignore")

## DATA LOADING

In [3]:
def read_instance(file_name):
        
    global J, M, Rmax
    
    with open(file_name,"r") as file:
        data = file.readlines()
    
    # PROBLEM SPECIFICATIONS: Line #1: number of machines (M), and of jobs (J)
    specs = data[0].split()
    J = int(specs[0])
    M = int(specs[1])

    machine_names = data[1].split()[0::2]

    prec_lines = len(data) - (J+1)*2 - 2 # Number of lines for precedences in the instance file

    # PROCESSING TIMES: lines # 2 ... 2+J-1
    skiprows_times = 1
    skipfooter_times = prec_lines + (J + 3)
    processing_times_df = pd.read_csv(file_name,
                                      sep="\s+",
                                      header=None,
                                      names=machine_names,
                                      index_col=False,
                                      usecols=list(range(1,2*M,2)),
                                      dtype=np.int8,
                                      engine='python',
                                      skiprows=skiprows_times,
                                      skipfooter=skipfooter_times
                                     )#.transpose()

    # RESOURCES: lines # 2 + J ... 2 * (J + 1)
    skiprows_res = J + 2
    skipfooter_res = prec_lines + 2
    resources_df = pd.read_csv(file_name,
                               sep="\s+",
                               header=None,
                               names=machine_names,
                               index_col=False,
                               usecols=list(range(1,2*M,2)),
                               dtype=np.int8,
                               engine='python',
                               skiprows=skiprows_res,
                               skipfooter=skipfooter_res
                              )#.transpose()
    
    Rmax = int(data[(J+1)*2 + 1])
    
    # PRECEDENCES: lines # 2 + J ... 2 * (J + 1)
    if prec_lines > 1: # Si 1, significaría que 'Precedencia' sería un campo vacío
        prec_array = np.full(shape=(J,), fill_value=None)
        for p in range(len(data)-prec_lines+1,len(data)):
            prec_line = data[p]
            prec_line = prec_line.split(':')
            prec_array[int(prec_line[0])] = int(prec_line[1])
        
    problem_df = pd.concat([processing_times_df, resources_df, pd.Series(prec_array)], axis=1, join='outer', copy=False)    
    
    print(f'\nInstance specifications: J = {J} jobs, M = {M} machines, prec = {prec_lines-1} precedence relationship(s), and Rmax = {Rmax} resources.')

    return problem_df


## DATA STRUCTURING

In [4]:
class Job:
    def __init__(self, index, p_times, res, prec):
        self.index = int(index) # Index (name) of the job as an int
        self.p_times = p_times # Processing times on each machine as a NumPy array of float
        self.res = res # Job resources on each machine as a NumPy array of float
        self.prec = prec # Previous job (with a precedence relation) as an int
        self.cost = int(0) # Job's assigned cost


In [5]:
class Machine:
    def __init__(self, index):
        self.index = int(index) # Index (name) of the machine as an int
        self.job_seq = np.empty(shape=(0)) #, dtype=np.int8) # Job sequence on the machine as a NumPy array
        self.job_times = np.empty(shape=(0), dtype=np.int8) # Job processing time on the machine as a NumPy array
        self.s_times = np.empty(shape=(0), dtype=np.int8) # Job processing time on the machine as a NumPy array
        self.e_times = np.empty(shape=(0), dtype=np.int8) # Job processing time on the machine as a NumPy array
        self.t_spans = {}
        self.job_res = np.empty(shape=(0), dtype=np.int8) # Job resources on the machine at every instant as a NumPy array
        self.C = int(0) # Machine makespan as an int
        
    def reset(self):
        self.job_seq = np.empty(shape=(0)) #, dtype=np.int8) # Job sequence on the machine as a NumPy array
        self.job_times = np.empty(shape=(0), dtype=np.int8) # Job processing time on the machine as a NumPy array
        self.s_times = np.empty(shape=(0), dtype=np.int8) # Job processing time on the machine as a NumPy array
        self.e_times = np.empty(shape=(0), dtype=np.int8) # Job processing time on the machine as a NumPy array
        self.t_spans.clear()
        self.job_res = np.empty(shape=(0), dtype=np.int8) # Job resources on the machine as a NumPy array
        self.C = int(0) # Machine makespan as an int
        
    def program_job(self, job, pos=-1):
        # Every parameter's length will be:
            # s_times, e_times, job_times, job_seq: J
            # job_res: C-1
        if pos == -1:
            job_s_time = int(0) if (self.job_seq.size == 0) else int(self.C)
            job_e_time = job_s_time + job.p_times[self.index]
            self.s_times = np.append(self.s_times, job_s_time)
            self.e_times = np.append(self.e_times, job_e_time)
            self.job_times = np.append(self.job_times, job.p_times[self.index])
            self.job_res = np.append(self.job_res, job.res[self.index]*np.ones(shape=(job.p_times[self.index],)))
            self.job_seq = np.append(self.job_seq, job)
        else:
            job_s_time = int(0) if (self.job_seq.size == 0) else int(self.s_times[pos])
            job_e_time = job_s_time + job.p_times[self.index]
            self.s_times = np.concatenate((self.s_times[:pos], 
                                           np.array([job_s_time]), 
                                           self.s_times[pos:] + job.p_times[self.index]))
            self.e_times = np.concatenate((self.e_times[:pos], 
                                           np.array([job_e_time]), 
                                           self.e_times[pos:] + job.p_times[self.index]))
            self.job_times = np.concatenate((self.job_times[:pos], 
                                             np.array([job.p_times[self.index]]), 
                                             self.job_times[pos:]))
            self.job_res = np.concatenate((self.job_res[:job_s_time], 
                                             job.res[self.index]*np.ones(shape=(job.p_times[self.index],)), 
                                             self.job_res[job_s_time:]))
            self.job_seq = np.concatenate((self.job_seq[:pos], 
                                           np.array([job]), 
                                           self.job_seq[pos:]))
        # print(f'\nprec job J{job.index}: {job.p_times}, {job.res}, {job.prec}, {job.cost}')
        # print(f'\ndep job pos: {pos} {type(pos)}')
        # print(f'\nmachine_seq.size = {self.job_seq.size} o {len(self.job_seq)}')
        # print(f'\nlen(machine_s_times) = {len(self.s_times)}')
        # print(f'\nmachine_s_times = {self.s_times}, {len(self.s_times)}')
        # print(f'\nmachine_e_times = {self.e_times}, {len(self.e_times)}')
        self.t_spans.clear()
        for j in range(len(self.job_seq)):
            self.t_spans.update({self.job_seq[j].index: (self.s_times[j], self.e_times[j])})
        # print(f'\nself.t_spans: {len(self.t_spans)}')
        self.C = self.e_times[-1] # Machine makespan as an int
        # return self.job_times
        

In [6]:
def create_jobs(problem_df):
    
    # Creation of the job list
    
    job_list = []

    for j in range(J):
        index = j
        p_times = np.array(list(problem_df.iloc[j,:M]))
        res = np.array(list(problem_df.iloc[j,M:2*M]))
        prec = int(problem_df.iloc[j,2*M]) if isinstance(problem_df.iloc[j,2*M], int) else problem_df.iloc[j,2*M]
        job = Job(index, p_times, res, prec)
        job_list.append(job)               

    return job_list    
    

In [7]:
def create_machines(num_machines):
    
    # Creation of the machine list
    
    machine_list = [Machine(m) for m in range(num_machines)]

    return machine_list
        

## DATA VISUALISATION

In [8]:
def display_solution(solution):
    
    # Display of a solution
    
    for machine in solution:
        print(f'\nMachine M{machine.index}:')
        print(f'Job sequence: \t\t {[job.index for job in machine.job_seq]}')
        print(f'Start times: \t\t {machine.s_times}')
        print(f'Processing times: \t {machine.job_times}')
        print(f'End times: \t\t {machine.e_times}')
        print(f'Resources: \t\t {machine.job_res}')
        print(f'Makespan: \t\t C = {machine.C}')

    C_max = max([machine.C for machine in solution])
    print(f'\nThe maximum makespan is C_max: {C_max}')
    
    return None

## SOLUTION FEASIBILITY

In [9]:
def assess_feasibility(solution):
        
# def assess_feasibility(solution):
    # Assesses whether a solution to the problem is feasible or unfeasible. Along with this condition, the function returns:
        # If feasible, sol_cost = sol_Cmax.
        # If unfeasible, sol_cost = sol_Cmax + (e_time_prec - s_time_dep) + abs(np.sum(acc_res - Rmax_res)).

    # time_span_dict_m = {}
    job_finder_dict = {}
    dependency_dict = {}    

    sol_Cmax = max([machine.C for machine in solution])
    res_matrix = np.zeros(shape=(M,sol_Cmax))
    sol_cost = sol_Cmax

    prec_ok = True
        
    _ = [job_finder_dict.update({job.index: machine.index}) for machine in solution for job in machine.job_seq]
    
    # print(f'job_finder_dict in assess: {job_finder_dict}, len={len(job_finder_dict.keys())}')

    for machine in solution:
        
    # We iterate over every machine and create:
        # For every job: an entry into the time_span_dict: key=job index, value=(start time, end time)
        # For every job with a precedence relationship: an entry into the dependency_dict: key=dependent job's index, value=precedent job's index                
        # For every job: a row in the res_matrix with the resource distribution over time (the whole time span of every machine)
            
        for pos in range(len(machine.job_seq)):
            
            # time_span_dict_m.update({machine.job_seq[pos].index : (machine.s_times[pos], machine.e_times[pos])})
            if isinstance(machine.job_seq[pos].prec, int):
                # dependency_dict.update({machine.job_seq[pos].index : machine.job_seq[pos].prec})
                # if not isinstance(machine.job_seq[pos].index, int):
                #     print(f'Assess: machine.job_seq[pos].index: {machine.job_seq[pos].indexb}')
                #     print(f'job_finder_dict in assess: {job_finder_dict}, len={len(job_finder_dict.keys())}')
                dep_job = machine.job_seq[pos].index
                # if not isinstance(machine.job_seq[pos].index, int):
                #     print(f'Assess: machine.job_seq[pos].index: {machine.job_seq[pos].index}')
                #     print(f'job_finder_dict in assess: {job_finder_dict}, len={len(job_finder_dict.keys())}')
                dep_machine_idx = machine.index
                # if not isinstance(machine.job_seq[pos].prec, int):
                #     print(f'Assess: machine.job_seq[pos].prec: {machine.job_seq[pos].prec}')
                #     print(f'job_finder_dict in assess: {job_finder_dict}, len={len(job_finder_dict.keys())}')
                prec_job = machine.job_seq[pos].prec
                # if not isinstance(job_finder_dict.get(prec_job), int):
                #     print(f'Assess: job_finder_dict.get(prec_job): {job_finder_dict.get(prec_job)}')
                #     print(f'job_finder_dict in assess: {job_finder_dict}, len={len(job_finder_dict.keys())}, prec_job: {prec_job}')
                #     display_solution(solution)
                prec_machine = job_finder_dict.get(prec_job)
                dependency_dict.update({(dep_machine_idx,dep_job) : (prec_machine,prec_job)})

        res_matrix[machine.index, :machine.C] = np.transpose(machine.job_res)

    # Precedence relationships are analysed:
        # Jobs with precedence relationships have to start once the precedent jobs have finalised.
        # Otherwise, the prec_ok condition is set to False.
        
    for dep_job_info,prec_job_info in dependency_dict.items():
        # s_time_dep = time_span_dict_m[dep_job][0]
        # e_time_prec = time_span_dict_m[prec_job][1]

        dep_machine = solution[dep_job_info[0]]
        prec_machine = solution[prec_job_info[0]]
        
        s_time_dep = dep_machine.t_spans.get(dep_job_info[1])[0]
        # e_time_dep = dep_machine.t_spans.get(dep_job_info[1])[1]
        # s_time_prec = prec_machine.t_spans.get(prec_job_info[1])[0]
        e_time_prec = prec_machine.t_spans.get(prec_job_info[1])[1]

        if s_time_dep < e_time_prec:
            prec_ok = False
            sol_cost += (e_time_prec - s_time_dep)
            break
            
    # Resource requirements are analysed:
        # Cumulative use of resources in all machines at every instant of time is calculated.
        # The res_ok condition is set to False as soon as Rmax is exceeded.
        
    acc_res = np.sum(res_matrix, axis=0)
    Rmax_res = Rmax*np.ones_like(acc_res)
    res_ok = True if np.all(acc_res <= Rmax) else False
    sol_cost += 0 if res_ok else abs(np.sum(acc_res - Rmax_res))
    
    return int(sol_cost), prec_ok, res_ok

# CONSTRUCTIVE PHASE

In [10]:
def construct_rcl(pre_sol, job_dict, rule, alpha):
    
    # Construction of a restricted candidate list (RCL) Pending jobs are sorted on every machine according to their cost value.

    rcl_dict = {}
    job_finder_dict = {}
            
    _ = [job_finder_dict.update({job.index: machine.index}) for machine in pre_sol for job in machine.job_seq]

    
    # Pending jobs are sorted on every machine according to their cost value.
    
    if rule == 'SPT':
        for m in range(M):
            for job in job_dict[m]:
                job.cost = job.p_times[m] + pre_sol[m].C
            rcl_dict[m] = sorted(job_dict[m], key=lambda x: x.p_times[m])
         
    elif rule == 'PREC':
        for m in range(M):
            p_time_avg = np.mean([job.p_times[m] for job in job_dict[m]])
            for job in job_dict[m]:
                job.cost = (job.p_times[m] + pre_sol[m].C) if not(isinstance(job.prec,int)) else (job.p_times[m] + pre_sol[m].C + p_time_avg)
            rcl_dict[m] = sorted(job_dict[m], key=lambda x: x.cost)

#     elif rule == 'RES':
#         for m in range(M):
#             p_time_avg = np.mean([job.p_times[m] for job in job_dict[m]])
#             for job in job_dict[m]:
#                 if isinstance(job.prec,int):
#                     job.cost = job.p_times[m] + pre_sol[m].C + p_time_avg + job.res[m]**2
#                     prec_job_idx = job.prec
                    
#                     prec_machine_idx = job_finder_dict.get(job.prec)
#                     pre_sol[prec_machine_idx].job_seq[prec_job_idx].cost = 
#                     job.cost = job.p_times[m] + pre_sol[m].C + p_time_avg + job.res[m]**2
#                 else:
#                     job.cost = (job.p_times[m] + pre_sol[m].C + job.res[m]**2)
#             rcl_dict[m] = sorted(job_dict[m], key=lambda x: x.cost)
            
    elif rule == 'RES':
        for m in range(M):
            p_time_avg = np.mean([job.p_times[m] for job in job_dict[m]])
            for job in job_dict[m]:
                job.cost = (job.p_times[m] + pre_sol[m].C + job.res[m]**2) if not(isinstance(job.prec,int)) else (job.p_times[m] + pre_sol[m].C + p_time_avg + job.res[m]**2)
            rcl_dict[m] = sorted(job_dict[m], key=lambda x: x.cost)

    
    cost_list = [[job.cost for job in job_list_k] for k,job_list_k in job_dict.items()] # List of lists containing the pending jobs' costs on every machine
    cost_array = np.asarray(cost_list)
    
    # Range of cost values to construct the RCL as a function of alpha.

    cost_min = min(np.min(cost_array, axis=1))
    cost_max = cost_min + alpha * (max(np.max(cost_array, axis=1)) - cost_min)
    
    # Pending jobs outside the limits of the RCL are removed.

    for k,job in rcl_dict.items():
        kj = len(rcl_dict[k]) - 1
        while kj >= 0 and (job[kj].cost > cost_max):
            rcl_dict[k].pop()
            kj -= 1

    return rcl_dict


In [11]:
def select_candidate(rcl_dict):
    
    candidate_machine_index = None
    candidate_job_index = None
    
    candidate_machine_list = [machine_index for machine_index in rcl_dict.keys() if len(rcl_dict[machine_index]) > 0]
    candidate_machine_index = random.choice(candidate_machine_list)
    candidate_job_index = random.choice(rcl_dict[candidate_machine_index]).index

    return candidate_machine_index, candidate_job_index


In [12]:
def construct_initial_solution(job_list, rule, alpha):

    # Creation of a dictionary with items defined by:
        # keys: indices of the machines
        # values: list of jobs sorted by the SPT rule (processing times in non-decreasing order on each machine)    
    
    job_dict = {}
    for m in range(M):
        # job_dict[m] = copy.deepcopy(job_list)
        job_dict.update({m: copy.deepcopy(job_list)})
    #     print(f'\n{len(job_dict[m])} jobs in machine {m}')
    #     x = [j.index for j in job_dict[m]]
    #     print(x)
    # pending_jobs = np.sum([len(v) for k,v in job_dict.items()], dtype=np.int8) // M # Number of jobs pending to be assigned
    pending_jobs = J
    # print(f'pending_jobs: {pending_jobs}')
    
    machine_env = create_machines(M)
    _ = [machine.reset() for machine in machine_env]
    
    # machine_env = []
    while pending_jobs > 0:
        
        rcl_dict = construct_rcl(machine_env, job_dict, rule, alpha)        
        candidate_machine_index, candidate_job_index = select_candidate(rcl_dict)
        # candidate_machine_index, candidate_job_index = select_candidate_old(rcl_dict, 'RND')
        # print(f'candidate_machine_index, candidate_job_index: {candidate_machine_index}, {candidate_job_index}')
        
        # Assign the SPT job to the corresponding machine and remove it from the rest
        for machine_index,jobs in job_dict.items():            
            # job_pos = np.where([job.index for job in jobs] == job_spt_index) #[0][0]
            job_pos = [job.index for job in jobs].index(candidate_job_index) #[0][0]
            # print(f'for: machine {machine_index}, jobs: {[job.index for job in jobs]}, candidate_job_index: {candidate_job_index}, job_pos: {job_pos}')
            candidate_job = jobs.pop(job_pos)
            if machine_index == candidate_machine_index:
                machine_env[machine_index].program_job(candidate_job)
       
        pending_jobs -= 1
        # pending_jobs = np.sum([len(v) for k,v in job_dict.items()], dtype=np.int8) // M
        # print(f'pending_jobs: {pending_jobs}')
        
    initial_Cmax, initial_prec_ok, initial_res_ok  = assess_feasibility(machine_env)
    initial_feasibility = initial_prec_ok and initial_res_ok

    return machine_env, initial_Cmax, initial_prec_ok, initial_res_ok

# LOCAL SEARCH PHASE

In [13]:
def external_insertion(initial_solution, initial_Cmax, initial_feasibility, insert_time_limit):
    
    # Investigate neighbourhood of the initial solution (i.e. machine_environment generated in the constructive phase)    
    
    insert_time = 0
    insert_start_time = time.monotonic()
    
    insert_time_exceeded = False
    
    sol_counter = 0
    
    if initial_feasibility:
        best_f_solution = copy.deepcopy(initial_solution)
        best_f_Cmax = initial_Cmax
        partial_prec_ok = True
        partial_res_ok = True
        best_u_solution = []
        best_u_Cmax = np.inf
    else:
        best_f_solution = []
        best_f_Cmax = np.inf
        best_u_solution = copy.deepcopy(initial_solution)
        best_u_Cmax = initial_Cmax
        partial_prec_ok = False
        partial_res_ok = False
        
    # print('\n' + ' EXTERNAL INSERTION '.center(160, '+'))
    
    for m0 in range(M):
        
        for j0 in range(len(initial_solution[m0].job_seq)):

            extracted_job_0 = initial_solution[m0].job_seq[j0]
                
            for m1 in (set(range(M))-set([m0])):

                # print(f'\nExternal insertion between machines M{m0} and M{m1}')
                # print('\n' + f' External insertion between machines M{m0} and M{m1} '.center(160, '-'))

                for j1 in range(len(initial_solution[m1].job_seq)+1):

                    partial_solution =  copy.deepcopy(initial_solution)

                    partial_solution_0 = np.concatenate((initial_solution[m0].job_seq[:j0], initial_solution[m0].job_seq[j0+1:]))
                    partial_solution[m0].reset()
                    for job in partial_solution_0:
                        partial_solution[m0].program_job(job)

                    partial_solution_1 = np.concatenate((initial_solution[m1].job_seq[:j1], [extracted_job_0], initial_solution[m1].job_seq[j1:]))
                    partial_solution[m1].reset()
                    for job in partial_solution_1:
                        partial_solution[m1].program_job(job)

                    sol_counter += 1
                    partial_Cmax, partial_prec_ok, partial_res_ok = assess_feasibility(partial_solution)
                    partial_feasibility = partial_prec_ok and partial_res_ok
    
                    if partial_feasibility:
                        if partial_Cmax < best_f_Cmax:
                            best_f_solution = copy.deepcopy(partial_solution)
                            best_f_Cmax = partial_Cmax
                            # print('Improved best_f_solution')
                    else:
                        if partial_Cmax < best_u_Cmax:
                            best_u_solution = copy.deepcopy(partial_solution)
                            best_u_Cmax = partial_Cmax
                            # print('Improved best_u_solution')
                            
                    del partial_solution
                    
                    insert_end_time = time.monotonic()
                    insert_time = insert_end_time - insert_start_time
                    insert_time_exceeded = insert_time >= insert_time_limit
                    
                    if insert_time_exceeded:
                        # print('insert break level 4')
                        break
                        
                if insert_time_exceeded:
                    # print('insert break level 3')
                    break
                    
            if insert_time_exceeded:
                # print('insert break level 2')
                break
                
        if insert_time_exceeded:
            # print('insert break level 1')
            break
                
    return partial_prec_ok, partial_res_ok, best_f_solution, best_f_Cmax, best_u_solution, best_u_Cmax, sol_counter


In [14]:
def external_swap(initial_solution, initial_Cmax, initial_feasibility, swap_time_limit):
    
    # Investigate neighbourhood of the initial solution (i.e. machine_environment generated in the constructive phase)
    # The looping structure of the internal insertion is modified to avoid assessing twice the same swaps (e.g., 0<->1, 1<->0)
    
    # Timers
    swap_time = 0
    swap_start_time = time.monotonic()    
    swap_time_exceeded = False
    
    # Solution counters
    sol_counter = 0
    
    if initial_feasibility:
        best_f_solution = copy.deepcopy(initial_solution)
        best_f_Cmax = initial_Cmax
        partial_prec_ok = True
        partial_res_ok = True
        best_u_solution = []
        best_u_Cmax = np.inf
    else:
        best_f_solution = []
        best_f_Cmax = np.inf
        best_u_solution = copy.deepcopy(initial_solution)
        best_u_Cmax = initial_Cmax
        partial_prec_ok = False
        partial_res_ok = False
        
    # print('\n' + ' EXTERNAL SWAP '.center(160, '+'))
    
    for m in range(M-1):
        
        # Generation of new solutions by means 2-opt operator. For J jobs, (J-1)x(J-2)/2 new solutions are generated
        
        m0 = m
        
        for j0 in range(len(initial_solution[m0].job_seq)):
            
            m1 = m0 + 1
            
            while m1 < M:
                
                # print('\n' + f' External swap between machines M{m0} and M{m1} '.center(160, '-'))
                        
                for j1 in range(len(initial_solution[m1].job_seq)):

                    ########################################################## EXTERNAL SWAP ##########################################################
                
                    # print(f'\nExternal swap between machines M{m0} and M{m1}')
                    # print('\n' + f' External swap between machines M{m0} and M{m1} '.center(160, '-'))

                    partial_solution =  copy.deepcopy(initial_solution)
                    
                    extracted_job_0 = initial_solution[m0].job_seq[j0]
                    extracted_job_1 = initial_solution[m1].job_seq[j1]
                    
                    partial_solution_0 = np.concatenate((initial_solution[m0].job_seq[:j0], [extracted_job_1], initial_solution[m0].job_seq[j0+1:]))
                    partial_solution[m0].reset()
                    for job in partial_solution_0:
                        partial_solution[m0].program_job(job)

                    partial_solution_1 = np.concatenate((initial_solution[m1].job_seq[:j1], [extracted_job_0], initial_solution[m1].job_seq[j1+1:]))
                    partial_solution[m1].reset()
                    for job in partial_solution_1:
                        partial_solution[m1].program_job(job)
                    
                    sol_counter += 1
                    partial_Cmax, partial_prec_ok, partial_res_ok = assess_feasibility(partial_solution)
                    partial_feasibility = partial_prec_ok and partial_res_ok

                    if partial_feasibility:
                        if partial_Cmax < best_f_Cmax:
                            best_f_solution = copy.deepcopy(partial_solution)
                            best_f_Cmax = partial_Cmax
                            # print('Improved best_f_solution')
                    else:
                        if partial_Cmax < best_u_Cmax:
                            best_u_solution = copy.deepcopy(partial_solution)
                            best_u_Cmax = partial_Cmax
                            # print('Improved best_u_solution')
                            
                    del partial_solution
                    
                    swap_end_time = time.monotonic()
                    swap_time = swap_end_time - swap_start_time
                    swap_time_exceeded = swap_time >= swap_time_limit
                    
                    if swap_time_exceeded:
                        # print('swap break level 4')
                        break
                        
                m1 += 1
                if swap_time_exceeded:
                    # print('swap break level 3')
                    break

            if swap_time_exceeded:
                # print('swap break level 2')
                break

        if swap_time_exceeded:
            # print('swap break level 1')
            break
                
    return partial_prec_ok, partial_res_ok, best_f_solution, best_f_Cmax, best_u_solution, best_u_Cmax, sol_counter


In [15]:
def local_search(initial_solution, initial_Cmax, initial_prec_ok, initial_res_ok, local_time, local_time_limit, search_iter_limit, search_stagnation_limit):
    
    # Local search algorithm    
    
    # Timers
    local_time = 0    
    local_start_time = time.monotonic()
    
    local_iter = 0
    search_improvement = False
    search_stagnation = 0
    
    best_feasible_dict = {}
    best_unfeasible_dict = {}
    
    # Solution counters
    local_sol_counter = 0
    insert_counter = 0
    swap_counter = 0

    partial_prec_ok = initial_prec_ok
    partial_res_ok = initial_res_ok
    initial_feasibility = initial_prec_ok and initial_res_ok

    if initial_feasibility:
        best_f_Cmax = initial_Cmax
        best_feasible_dict.update({initial_Cmax: copy.deepcopy(initial_solution)})
        best_u_Cmax = np.inf
    else:
        best_f_Cmax = np.inf
        best_u_Cmax = initial_Cmax
        best_unfeasible_dict.update({initial_Cmax: copy.deepcopy(initial_solution)})
    
    while (local_time < local_time_limit) and (search_stagnation < search_stagnation_limit): # and (local_iter < search_iter_limit): 
        
        # print(f'Local search iteration No. {local_iter} starting at {round(local_time, 4)} / {local_time_limit} s        ', end='\r')
        
        
        # External insertion
        insert_time_limit = (local_time_limit - local_time) / 2
        # print(f'insert_time_limit {insert_time_limit}')
        partial_prec_ok, partial_res_ok, partial_f_sol, partial_f_Cmax, partial_u_sol, partial_u_Cmax, insert_counter = external_insertion(initial_solution, 
                                                                                                                        initial_Cmax, 
                                                                                                                        initial_feasibility, 
                                                                                                                        insert_time_limit)
        local_sol_counter += insert_counter
        # Update solutions, if necessary. With the following structure, we prioritise feasible solutions (with respect to unfeasible ones)
        if (len(partial_u_sol) > 0) and (partial_u_Cmax < best_u_Cmax):
            best_unfeasible_dict.update({partial_u_Cmax: partial_u_sol})
            best_u_Cmax = partial_u_Cmax
            initial_solution = partial_u_sol
            initial_Cmax = partial_u_Cmax
            initial_feasibility = False
            search_improvement = True
        if (len(partial_f_sol) > 0) and (partial_f_Cmax < best_f_Cmax):
            best_feasible_dict.update({partial_f_Cmax: partial_f_sol})
            best_f_Cmax = partial_f_Cmax
            initial_solution = partial_f_sol
            initial_Cmax = partial_f_Cmax
            initial_feasibility = True
            search_improvement = True

        # External swap
        swap_time_limit = (local_time_limit - local_time) / 2
        # print(f'swap_time_limit {swap_time_limit}')
        partial_prec_ok, partial_res_ok, partial_f_sol, partial_f_Cmax, partial_u_sol, partial_u_Cmax, swap_counter = external_swap(initial_solution, 
                                                                                                                                    initial_Cmax, 
                                                                                                                                    initial_feasibility, 
                                                                                                                                    swap_time_limit)
        local_sol_counter += swap_counter
        # Update solutions, if necessary. With the following structure, we prioritise feasible solutions (with respect to unfeasible ones)
        if (len(partial_u_sol) > 0) and (partial_u_Cmax < best_u_Cmax):
            best_unfeasible_dict.update({partial_u_Cmax: partial_u_sol})
            best_u_Cmax = partial_u_Cmax
            initial_solution = partial_u_sol
            initial_Cmax = partial_u_Cmax
            initial_feasibility = False
            search_improvement = True
        if (len(partial_f_sol) > 0) and (partial_f_Cmax < best_f_Cmax):
            best_feasible_dict.update({partial_f_Cmax: partial_f_sol})
            best_f_Cmax = partial_f_Cmax
            initial_solution = partial_f_sol
            initial_Cmax = partial_f_Cmax
            initial_feasibility = True
            search_improvement = True
            
        if not search_improvement:
            search_stagnation += 1
        else:
            search_stagnation = 0
            search_improvement = False
        
        # print((f'\nSearch improvement' if (search_stagnation == 0) else f'\nNo search improvement') + (f'at iteration No. {local_iter}'))
        
        local_iter += 1

        local_end_time = time.monotonic()
        local_time += (local_end_time - local_start_time)
        
    # local_iter -= 1 # Update to true local_iter value
    best_prec_ok = partial_prec_ok
    best_res_ok = partial_res_ok
        
    # print((f'\nExit local search after {local_iter} iterations with search_iter_limit = {search_iter_limit} and search_stagnation_limit = {search_stagnation_limit}.'))
        
    return best_prec_ok, best_res_ok, best_feasible_dict, best_unfeasible_dict, local_iter


## SOLUTION REPAIRING

In [16]:
def internal_swap(unsorted_sol, m_index, dep_job_index, prec_job_index):
    
    # This function is called only when a dependent job is processed before its precedent job 
    sorted_sol = copy.deepcopy(unsorted_sol)
    sorted_sol[m_index].reset()
    
    # unsorted_machine = unsorted_sol[m_index]
    unsorted_job_list = [job for job in unsorted_sol[m_index].job_seq]
    unsorted_job_list_indices = [job.index for job in unsorted_job_list]
    # print(f'\nunsorted_job_list_indices: {unsorted_job_list_indices}, {type(unsorted_job_list_indices)}')
    dep_job_pos = unsorted_job_list_indices.index(dep_job_index)
    prec_job_pos = unsorted_job_list_indices.index(prec_job_index)

    # Firstly, jobs before the dependent one are re-programmed as they were
    # for i in unsorted_job_list_indices:
    for pos, idx in enumerate(unsorted_job_list_indices):
        # print(f'\ni: {i}')
        
        if idx == dep_job_index:
            # print(f'\n1. len(unsorted_job_list): {len(unsorted_job_list)}, p: {p}, job: {unsorted_job_list[p].index}')
            # sorted_sol[m_index].program_job(unsorted_job_list[prec_job_pos], p)
            p = prec_job_pos
            # j = unsorted_job_list[prec_job_pos]
        
        elif idx == prec_job_index:
            # Secondly, the precedent job is programmed where the dependent job started
            # print(f'2. len(unsorted_job_list): {len(unsorted_job_list)}, unsorted_job_list[prec_job_pos]: {unsorted_job_list[prec_job_pos].index}, dep_job_pos: {dep_job_pos}')
            # sorted_sol[m_index].program_job(unsorted_job_list.(dep_job_pos))
            p = dep_job_pos
            # j = unsorted_job_list[dep_job_pos]
            # unsorted_job_list.remove(unsorted_job_list[prec_job_pos])
            
        else:
    
    # Thirdly, the remaining jobs following, and including, the dependent one are re-programmed ass they were
    # for p in range(dep_job_pos, len(unsorted_job_list)):
        # print(f'3. len(unsorted_job_list): {len(unsorted_job_list)}, p: {p}, job: {unsorted_job_list[p].index}')
            # sorted_sol[m_index].program_job(unsorted_job_list[p])
            p = pos
        
        j = unsorted_job_list[p]
            
        sorted_sol[m_index].program_job(j)
    
    return sorted_sol

In [17]:
def repair(unfeasible_dict, repaired_prec_ok, repaired_res_ok, repair_time_limit):
        
    # To repair an unfeasible solution:
        # Soluctions with precedence relationships are sought
        # Starting times times of those jobs are compared against the finishing times of the precedent jobs 
    
    repair_iter = 0
    repair_time = 0
    repair_prep_start_time = time.monotonic()
    
    job_finder_dict = {}
    time_span_dict_m = {}
    dependency_dict = {}
    dependency_list = []
    repaired_dict = {}
    
    best_u_Cmax = min(unfeasible_dict.keys())
    unfeasible_sol = unfeasible_dict[best_u_Cmax]
    repaired_sol = copy.deepcopy(unfeasible_sol)

    sol_Cmax = max([machine.C for machine in unfeasible_sol])  
    res_matrix = np.zeros(shape=(M,sol_Cmax))
    # sol_cost = sol_Cmax

    # Precedence relationships are stored in a dictionary (dependency_dict) with the following info:
        # key: a tuple comprising the dependent job's machine index and job index
        # value: a tuple comprising a boolean stating whether both jobs are in the same machine, and the precedent job's index
    # Parallelly, jobs' start and end times are stored as tuples in another dictionary (time_span_dict_m)
    # Finally, the resources required by every job at every instant of time are stored in a matrix (res_matrix)

    # for machine in unfeasible_sol:
        
    _ = [job_finder_dict.update({job.index: machine.index}) for machine in unfeasible_sol for job in machine.job_seq]

    # print(f'job_finder_dict in repair: {job_finder_dict}, len={len(job_finder_dict.keys())}')

    for machine in unfeasible_sol:

        # job_indices = [j.index for j in machine.job_seq]

        for pos in range(len(machine.job_seq)):

            # time_span_dict_m[machine.job_seq[pos].index] = (machine.s_times[pos], machine.e_times[pos])

            if isinstance(machine.job_seq[pos].prec, int):
                dep_job = machine.job_seq[pos].index
                dep_machine_idx = machine.index
                # print(f'Repair: dep_machine_idx: {dep_machine_idx}, dep_job: {dep_job}')
                prec_job = machine.job_seq[pos].prec
                # print(f'Repair: prec_job: {prec_job}')
                prec_machine = job_finder_dict.get(prec_job)
                # print(f'Repair: prec_machine: {prec_machine}')
                dependency_dict[(dep_machine_idx,dep_job)] = (prec_machine,prec_job)

                # A list of lists (dependency_list) is created to store interdependencies:
                    # In the simplest case, every item (list) will contain two elements: dep_job, prec_job
                    # In more complex situations, every item (list) will contain multiple elements: dep_job, prec_job_1, prec_job_2...

                dependency_link = False

                for s in range(len(dependency_list)):
                    if dep_job in dependency_list[s]:
                        # print(f'dep_job {dep_job} in dependency_list[{s}]')
                        dependency_list[s].append(prec_job)
                        dependency_link = True
                    elif prec_job in dependency_list[s]:
                        # print(f'prec_job {prec_job} in dependency_list[{s}]')
                        dependency_list[s].insert(0, dep_job)
                        dependency_link = True

                if not dependency_link:
                    dependency_list.append([dep_job, prec_job])
                    dependency_link = False                    

        res_matrix[machine.index, :machine.C] = np.transpose(machine.job_res)

    # A reversed version of the dependency_list is created:
        # The first element [0] in every item (list) is the firt precedent job which will not be shifted
        # The following elements [1:] in every item (list) will be ncrementally shifted

    reversed_dependency_list = copy.deepcopy(dependency_list)
    for item in reversed_dependency_list:
        item.reverse()
    # print(f'dependency_dict: {dependency_dict}')

    # Precedence relationships are revised:
        # If both jobs are in the same machine, and the dependent job is processed before the precedent job, they are immediately swapped
        # In other cases, the precedence relationships will be fixed afterwards, whenever necessary

    # The following processing would not be necessary if only simple precedence relationships existed. 
    # However, it becomes necessary when complex precedence relationships may arise.

    dep_job_info_list = list(dependency_dict.keys()) # list of tuples (machine, dep_job)
    # print(f'dep_job_info_list2: {dep_job_info_list2}, {type(dep_job_info_list2)}')    

    idle_job = Job(index=int(J), p_times=np.ones(shape=(M,), dtype=np.int8), res=np.zeros(shape=(M,), dtype=np.int8), prec=None)
    
    repair_prep_end_time = time.monotonic()
    repair_prep_time = (repair_prep_end_time - repair_prep_start_time)
    repair_time += repair_prep_time
    
    repaired_sol_feasibility = repaired_prec_ok and repaired_res_ok
    
    print('')

    while (repair_time < repair_time_limit) and (not repaired_sol_feasibility):

        repair_iter += 1
        
        print(f'Repair iteration starting at {round(repair_time, 4)} / {repair_time_limit} s        ', end='\r')
        
        if not repaired_prec_ok:

            ########################################################## PRECEDENCE RELATIONSHIPS ##########################################################

            # print('\n' + f' PRECEDENCE RELATIONSHIPS '.center(160, '+')) # 483

            repair_prec_start_time = time.monotonic()
            
            for dep_seq in reversed_dependency_list:

                # print(f'\nEnter for dep_seq: {dep_seq}')

                for i in dep_seq[1:]:

                    # print(f'\nEnter for i in dep_seq: {i}')

                    pos_list = [True if i == dep_job_tuple[1] else False for dep_job_tuple in dep_job_info_list]
                    pos = pos_list.index(True)
                    # print(f'{i} in dep_seq in dep_job_info_list in {dep_job_info_list[pos]}')

                    dep_job_info = dep_job_info_list[pos]
                    prec_job_info = dependency_dict.get(dep_job_info)
                    # print(f'dep_job_info: {dep_job_info}, prec_job_info: {prec_job_info}')
                    
                    dep_machine = repaired_sol[dep_job_info[0]]
                    prec_machine = repaired_sol[prec_job_info[0]]
                    # print(f'dep_machine_idx: {dep_machine.index}, prec_machine_idx: {prec_machine.index}')
                    
                    # if dep_machine.index == prec_machine.index and prec_machine.index == 13:
                    #     display_solution(repaired_sol)
                    # print(f'dep_machine.t_spans.keys: {dep_machine.t_spans.keys()}')    
                    
                    s_time_dep = dep_machine.t_spans.get(dep_job_info[1])[0]
                    e_time_dep = dep_machine.t_spans.get(dep_job_info[1])[1]
                    s_time_prec = prec_machine.t_spans.get(prec_job_info[1])[0]
                    e_time_prec = prec_machine.t_spans.get(prec_job_info[1])[1]
                    
                    # print(f's_time_dep: {s_time_dep}, e_time_dep: {e_time_dep}, s_time_prec: {s_time_prec}, e_time_prec: {e_time_prec}')
                    
                    if s_time_dep < e_time_prec:

                        if dep_machine == prec_machine:
                            # Swap the jobs when they are on the same machine
                            # print(f'\nPrecedence non compliance on the same machine M{dep_machine.index}')
                            # print(f'\nInternal swap in machine M{dep_job_info[0]} between dependent job J{dep_job_info[1]} and precedent job J{prec_job_info[1]}.')    
                            repaired_sol = internal_swap(repaired_sol, dep_job_info[0], dep_job_info[1], prec_job_info[1])
                        else:
                            # Shift the dependent job when it is on a different job then the precedent job
                            # print(f'\nPrecedence non compliance on different machines')
                            job_indices = [j.index for j in dep_machine.job_seq]
                            pos_dep_job = job_indices.index(dep_job_info[1])
                            # print(f'\ndep_machine: {dep_machine.index}, pos_dep_job: {pos_dep_job}')
                            # print(f'\nShift dependent job J{dep_job_info[1]} {e_time_prec-s_time_dep} processing unit times.')    
                            for d in range(e_time_prec-s_time_dep):
                                dep_machine.program_job(idle_job, int(pos_dep_job))                            

                        # Update the resource matrix, and ime spans in the corresponding dictionary:
                        sol_Cmax = max([machine.C for machine in repaired_sol])
                        res_matrix = np.zeros(shape=(M,sol_Cmax))

                        for machine in repaired_sol:

                            # for pos in range(len(machine.job_seq)):
                            #     # time_span_dict_m.[dep_machine.job_seq[pos].index] = (dep_machine.s_times[pos], dep_machine.e_times[pos])
                            #     time_span_dict_m.update({machine.job_seq[pos].index : (machine.s_times[pos], machine.e_times[pos])})
                                
                            res_matrix[machine.index, :machine.C] = np.transpose(machine.job_res)

                # print('\nRepaired prec:')
                # display_solution(repaired_sol)
                _, repaired_prec_ok, repaired_res_ok = assess_feasibility(repaired_sol)
                repaired_sol_feasibility = repaired_prec_ok and repaired_res_ok

                if repaired_prec_ok:
                    # print(f'\nbreak for i in dep_seq: {i} in level 1')
                    break
            
            repair_prec_end_time = time.monotonic()
            repair_prec_time = (repair_prec_end_time - repair_prec_start_time)
            repair_time += repair_prec_time
            
            # print(f'\nRepaired prec in solution at iter {repair_iter}, time {repair_prec_time}: feasibility: {repaired_sol_feasibility} (precedence compliance: {repaired_prec_ok}, and resource compliance {repaired_res_ok})')
            
        elif not repaired_res_ok:

            ############################################################## RESOURCES ##############################################################

            # Resources exceeded: This happens when a new job starts

            repair_res_start_time = time.monotonic()

            acc_res = np.sum(res_matrix, axis=0)
            repaired_res_ok = True if np.all(acc_res <= Rmax) else False

            p_x_res = np.where(acc_res > Rmax)[0][0] # Index where maximum resources are first exceeded. It will always be ind_x_res <= Cmax.

            candidate_machine_idx = [machine.index for machine in repaired_sol if (p_x_res in machine.s_times)]
            candidate_machine_C = [machine.C if (machine.index in candidate_machine_idx) else np.inf for machine in repaired_sol]
            candidate_machine_C_idxd = list(enumerate(candidate_machine_C))
            sorted_candidate_machine_C_idxd = sorted(candidate_machine_C_idxd, key=lambda x: x[1])

            m_idx = 0

            while (acc_res[p_x_res] > Rmax) and (sorted_candidate_machine_C_idxd[m_idx][1] < np.inf):
                idle_m_index = sorted_candidate_machine_C_idxd[m_idx][0]
                idle_j_pos = np.where(repaired_sol[idle_m_index].s_times == p_x_res)[0][0]
            
                repaired_sol[idle_m_index].program_job(idle_job, int(idle_j_pos))
                sol_Cmax = max([machine.C for machine in repaired_sol])
                
                res_matrix = np.zeros(shape=(M,sol_Cmax))
            
                for machine in repaired_sol:
    
                    res_matrix[machine.index, :machine.C] = np.transpose(machine.job_res)               
                
                acc_res = np.sum(res_matrix, axis=0)
            
                m_idx += 1
            
            _, repaired_prec_ok, repaired_res_ok = assess_feasibility(repaired_sol)
            repaired_sol_feasibility = repaired_prec_ok and repaired_res_ok
        
            repair_res_end_time = time.monotonic()
            repair_res_time = (repair_res_end_time - repair_res_start_time)
            repair_time += repair_res_time

    repaired_dict.update({sol_Cmax : repaired_sol})    

    return repaired_dict, repaired_prec_ok, repaired_res_ok

# MAIN PROGRAM

In [ ]:

################################################################ ALGORITHM CONFIGURATION ################################################################

os.chdir('C:\\Users\\Usuario\\notebooks\\09MIAR') # Home
# os.chdir('C:\\Users\\jbb\\Notebooks') # Work
parent_path = os.getcwd() # 'C:\\Users\\Usuario\\notebooks\\09MIAR'
# print(f'\nparent_path: {parent_path}')

debug_path = os.path.join(parent_path, 'input_data', 'debug')
cal_path = os.path.join(parent_pathh, 'input_data', 'cal')
test_small_path = os.path.join(parent_pathh, 'input_data', 'test_small')
test_large_path = os.path.join(parent_pathh, 'input_data', 'test_large')

instance_path = test_large_path
print(f'\ninstance_path: {instance_path}')

entries = os.listdir(instance_path)
# data_folders = [e for e in entries if (os.path.isdir(instance_path + '/' + e) and not(e.startswith('.ipynb')))]
# print(f'\ndata_fonlders: {data_folders}')
instances = [i for i in entries if (os.path.isfile(instance_path + '/' + i) and i.endswith('.txt'))]
print(f'\ninstances: {instances}')

os.chdir(instance_path)

if (instance_path == cal_debug_path) or (instance_path == test_small_path) or (instance_path == test_large_path):

    variant_dict = {'grasp0': (1.00, False), 'grasp1': (0.00, False), 'grasp2': (0.00, True), 'grasp3': (0.25, False), 'grasp4': (0.25, True)}

for var_idx, var_params in variant_dict.items():

    T_factor = 2
    problem_alpha = var_params[0]
    local_search_enabled = var_params[1]
            
    instance_sol = {}
    # instance_names = [i.rstrip('.txt') for i in instances]
    # print(f'\ninstance_path: {instance_path}')
    # instances = [i for i in entries if (os.path.isfile(instance_path + '/' + i) and i.endswith('.txt') and not('old' in i))]
    # print(f'\ninstances: {len(instances)}')

    for instance in instances:

        instance_name = instance.rstrip('.txt')

        print('\n\n' + str(' ' + instance_name + ' ').center(160, '#')) # 483

        ###################################################################### DATA LOADING ######################################################################

        load_start_time = time.monotonic()

        problem_df = read_instance(instance)

        load_end_time = time.monotonic()

        load_time = (load_end_time - load_start_time)

        # display(problem_df)

        ###################################################################### MULTI-START ######################################################################

        random.seed(42)
        g_rule = 'RES'

        search_iter_limit = J
        search_stagnation_limit = J if local_search_enabled else 0

        # Solution dictionaries
        feasible_sol_dict = {}
        repaired_sol_dict = {}
        best_grasp_f_dict = {}
        best_grasp_u_dict = {}
        unfeasible_sol_dict = {}

        best_grasp_f_Cmax = np.inf
        best_grasp_f_sol = []
        best_grasp_f_dict.update({best_grasp_f_Cmax: best_grasp_f_sol})

        best_grasp_u_Cmax = np.inf
        best_grasp_u_sol = []
        best_grasp_u_dict.update({best_grasp_u_Cmax: best_grasp_u_sol})

        instance_Cmax = np.inf

        # Iteration counters
        grasp_iter = 0
        grasp_improvement = False
        grasp_stagnation = 0

        # Solution counters
        grasp_sol_counter_cum = 0
        construct_sol_counter_cum = 0
        search_sol_counter_cum = 0
        repair_sol_counter_cum = 0
        best_sol_counter_cum = 0

        # Timers
        grasp_time = 0
        construct_time = 0
        search_time = 0
        repair_time = 0
        best_time = 0

        instance_T = float(J * M)
        iter_T = float(J * M) / T_factor
        print(f'\nInstance_time_limit: {instance_T}\nIter_time_limit: {iter_T}')
        print(f'\nConstructive phase: alpha = {problem_alpha}\nSearch phase: {bool(search_stagnation_limit)}\n')

        # problem_alpha = alpha_dict.get(alpha_index)

        best_start_time = time.monotonic()

        while grasp_time < instance_T: # and (grasp_stagnation < grasp_stagnation_limit):
        # while (grasp_iter < grasp_iter_limit): # and (grasp_stagnation < grasp_stagnation_limit):

            grasp_iter_start_time = time.monotonic()
            grasp_iter += 1

            print(f'GRASP iteration No. {grasp_iter} starting at {round(grasp_time, 4)} / {instance_T} s        ', end='\r')

            ################################################################## CONSTRUCTIVE PHASE ##################################################################

            # print('\n' + ' CONSTRUCTIVE PHASE '.center(160, '*'))

            construct_start_time = time.monotonic()

            job_list = create_jobs(problem_df)
            # machine_env = create_machines(M)
            # _ = [machine.reset() for# machine in machine_env]

            # Every GRASP iteration starts with its own initial solution regardless whether it improves or not the previous ones.
            # The purpose is to explore other areas of the solution space.

            initial_solution, initial_Cmax, initial_prec_ok, initial_res_ok = construct_initial_solution(job_list, g_rule, problem_alpha)
            initial_feasibility = initial_prec_ok and initial_res_ok
            # print(f'\nInitial solution: Cmax: {initial_Cmax}, feasibility: {initial_feasibility}')
            construct_sol_counter_cum += 1

            if (initial_feasibility) and (initial_Cmax < best_grasp_f_Cmax):
                feasible_sol_dict.update({initial_Cmax: initial_solution})
                best_sol_counter_cum = construct_sol_counter_cum + search_sol_counter_cum
                best_end_time = time.monotonic()
                best_time = (best_end_time - best_start_time)
            elif (not initial_feasibility) and (initial_Cmax < best_grasp_u_Cmax):
                unfeasible_sol_dict.update({initial_Cmax: initial_solution})        

            construct_end_time = time.monotonic()
            construct_time += (construct_end_time - construct_start_time)

            grasp_time += construct_time

            # display_solution(initial_solution)
            # print('\nTime elapsed during the constructive phase: {0:4.4f} (ms)\n'.format(construct_time * 1000))

            ################################################## lOCAL SEARCH PHASE ##################################################

            # print(' LOCAL SEARCH PHASE '.center(160, '*'))

            search_start_time = time.monotonic()

            neighbour_prec_ok, neighbour_res_ok, neighbour_feasible_dict, neighbour_unfeasible_dict, search_sol_counter = local_search(initial_solution, 
                                                                                                                                      initial_Cmax, 
                                                                                                                                      initial_prec_ok,
                                                                                                                                      initial_res_ok, 
                                                                                                                                      grasp_time, 
                                                                                                                                      iter_T, 
                                                                                                                                      search_iter_limit, 
                                                                                                                                      search_stagnation_limit)

            # If the local search returns any solutions, regardless being feasible and/or unfeasibles, they will improve Cmax and so we will add them to the dicts.
            feasible_sol_dict.update(neighbour_feasible_dict)
            unfeasible_sol_dict.update(neighbour_unfeasible_dict)
            search_sol_counter_cum += search_sol_counter

            if len(unfeasible_sol_dict) >= 1:

                best_u_Cmax = min(unfeasible_sol_dict.keys())

                if best_u_Cmax < best_grasp_u_Cmax:
                    best_grasp_u_Cmax = best_u_Cmax
                    best_grasp_u_sol = unfeasible_sol_dict.get(best_u_Cmax)
                    best_grasp_u_dict.clear() # Comment this line if the nuimber of unfeasible solutions should be reduced to one
                    best_grasp_u_dict.update({best_grasp_u_Cmax: best_grasp_u_sol})
                    best_grasp_prec_ok = neighbour_prec_ok
                    best_grasp_res_ok = neighbour_res_ok
                    # print('\n' + f' Partial Best GRASP Unfeasible Solution at Iteration No. {grasp_iter} '.center(483, '+'))
                    # display_solution(best_grasp_u_dict.get(best_grasp_u_Cmax))
                    grasp_improvement = True

            if len(feasible_sol_dict) >= 1:

                best_f_Cmax = min(feasible_sol_dict.keys())

                if best_f_Cmax < best_grasp_f_Cmax:
                    best_grasp_f_Cmax = best_f_Cmax
                    best_grasp_f_sol = feasible_sol_dict.get(best_f_Cmax)
                    best_grasp_f_dict.clear() # Comment this line if the nuimber of feasible solutions should be reduced to one
                    best_grasp_f_dict.update({best_grasp_f_Cmax: best_grasp_f_sol})
                    best_sol_counter_cum = construct_sol_counter_cum + search_sol_counter_cum
                    best_end_time = time.monotonic()
                    best_time = (best_end_time - best_start_time)
                    # print('\n' + f' Partial Best GRASP Feasible Solution at Iteration No. {grasp_iter} '.center(483, '+'))
                    # display_solution(best_grasp_f_dict.get(best_grasp_f_Cmax))
                    grasp_improvement = True

            if not grasp_improvement:
                grasp_stagnation += 1
            else:
                grasp_stagnation = 0
                grasp_improvement = False

            search_end_time = time.monotonic()
            search_time += (search_end_time - search_start_time)

            grasp_iter_end_time = time.monotonic()
            # grasp_time += (grasp_iter_end_time - grasp_iter_start_time)
            grasp_time += search_time

        ####################################################################### REPAIR #######################################################################

        if (len(best_grasp_f_dict) > 0) and (best_grasp_f_Cmax < np.inf):

            instance_Cmax = best_grasp_f_Cmax
            best_sol_prec_ok = True
            best_sol_res_ok = True
            best_sol_feasibility = best_sol_prec_ok and best_sol_res_ok
            repair_needed = False
            # print('\n\n' + f' Best GRASP Feasible Solution '.center(160, '*')) # 483
            # display_solution(best_grasp_f_dict.get(best_grasp_f_Cmax))
            # print(f'\n GRASP Solution: Cmax: {best_grasp_f_Cmax}, feasibility: {repaired_sol_feasibility} \
            #       (precedence compliance: {prec_ok}, and resource compliance {res_ok})')

        elif len(best_grasp_u_dict) > 0:

            # print('\n\n' + f' Best GRASP Repaired Solution '.center(160, '*')) # 483

            # print('\n\n' + f' Best GRASP Unfeasible Solution '.center(160, '*')) # 483
            # display_solution(best_grasp_u_dict.get(best_grasp_u_Cmax))

            repair_start_time = time.monotonic()
            best_grasp_r_dict, best_sol_prec_ok, best_sol_res_ok = repair(best_grasp_u_dict, best_grasp_prec_ok, best_grasp_res_ok, iter_T)
            # best_grasp_r_dict, best_sol_prec_ok, best_sol_res_ok = repair(best_grasp_u_dict, best_grasp_prec_ok, best_grasp_res_ok, 5)
            best_sol_feasibility = best_sol_prec_ok and best_sol_res_ok
            repair_end_time = time.monotonic()
            repair_time = (repair_end_time - repair_start_time)
            if best_sol_feasibility:
                best_sol_counter_cum = construct_sol_counter_cum + search_sol_counter_cum
                best_end_time = time.monotonic()
                best_time = (best_end_time - best_start_time)

            best_grasp_r_Cmax = list(best_grasp_r_dict.keys())[0]
            instance_Cmax = best_grasp_r_Cmax
            repair_needed = True
            # print('\n\n' + f' Best GRASP Repaired Solution '.center(160, '*')) # 483
            # display_solution(best_grasp_r_dict.get(best_grasp_r_Cmax))
        
        print(f'\nBest solution: Cmax: {instance_Cmax}, feasibility: {best_sol_feasibility} (precedence compliance: {best_sol_prec_ok}, and resource compliance {best_sol_res_ok})')
        grasp_sol_counter_cum = construct_sol_counter_cum + search_sol_counter_cum + repair_sol_counter_cum

        instance_sol.update({instance_name : [problem_alpha, 
                                              instance_T, 
                                              T_factor, 
                                              instance_Cmax, 
                                              best_sol_feasibility, 
                                              # best_sol_prec_ok, 
                                              # best_sol_res_ok, 
                                              repair_needed, 
                                              round(best_time, 4), 
                                              best_sol_counter_cum, 
                                              grasp_sol_counter_cum, 
                                              grasp_iter, 
                                              round(construct_time, 4), 
                                              round(search_time, 4), 
                                              round(repair_time, 4)]})

        # print(f'instance_sol_dict: {instance_sol[instance_name]}')

        #################################################################### FINAL EVALUATION ####################################################################


    print('\n\n' + ' SUMMARY '.center(160, '+')) #483

    column_names = ['Alpha', 'T_limit', 'T_factor', 'Makespan', 'Feasibility', 'Repair', 'Time2best',
                    'Sol2best', 'Total_sol', 'GRASP iter', 'Construct_time', 'Search_time', 'Repair_time']
    summary_df = pd.DataFrame.from_dict(instance_sol, orient='index', dtype=None, columns=column_names)

    display(summary_df)

    # os.chdir(parent_path)

    output_name = instance_path.replace(parent_path, '')
    output_name = output_name.lstrip('\ ')
    output_name = output_name + '_' + str(var_idx) + '.csv'

    output_path = os.path.join(parent_pathh, 'output_data', 'output_test_large', output_name)
    summary_df.to_csv(output_path, sep=';', header=True, index=True, mode='w', decimal=',')

    os.chdir(instance_path)

os.chdir(parent_path)



instance_path: C:\Users\Usuario\notebooks\09MIAR\test_large

instances: ['050x10_1-10_1_1.txt', '050x10_1-10_1_2.txt', '050x10_1-10_1_3.txt', '050x10_1-10_1_4.txt', '050x10_1-10_1_5.txt', '050x15_1-10_1_1.txt', '050x15_1-10_1_2.txt', '050x15_1-10_1_3.txt', '050x15_1-10_1_4.txt', '050x15_1-10_1_5.txt', '050x20_1-10_1_1.txt', '050x20_1-10_1_2.txt', '050x20_1-10_1_3.txt', '050x20_1-10_1_4.txt', '050x20_1-10_1_5.txt', '050x25_1-10_1_1.txt', '050x25_1-10_1_2.txt', '050x25_1-10_1_3.txt', '050x25_1-10_1_4.txt', '050x25_1-10_1_5.txt', '100x10_1-10_1_1.txt', '100x10_1-10_1_2.txt', '100x10_1-10_1_3.txt', '100x10_1-10_1_4.txt', '100x10_1-10_1_5.txt', '100x15_1-10_1_1.txt', '100x15_1-10_1_2.txt', '100x15_1-10_1_3.txt', '100x15_1-10_1_4.txt', '100x15_1-10_1_5.txt', '100x20_1-10_1_1.txt', '100x20_1-10_1_2.txt', '100x20_1-10_1_3.txt', '100x20_1-10_1_4.txt', '100x20_1-10_1_5.txt', '100x25_1-10_1_1.txt', '100x25_1-10_1_2.txt', '100x25_1-10_1_3.txt', '100x25_1-10_1_4.txt', '100x25_1-10_1_5.txt', '150x1

,Alpha,T_limit,T_factor,Makespan,Feasibility,Repair,Time2best,Sol2best,Total_sol,GRASP iter,Construct_time,Search_time,Repair_time
050x10_1-10_1_1,1.0,500.0,2,504,True,True,10.250,121,121,121,8.637,0.253,1.360
050x10_1-10_1_2,1.0,500.0,2,507,True,True,10.391,110,110,110,8.811,0.190,1.360
050x10_1-10_1_3,1.0,500.0,2,539,True,True,12.047,104,104,104,9.343,0.236,2.468
050x10_1-10_1_4,1.0,500.0,2,517,True,True,11.031,102,102,102,9.232,0.189,1.594
050x10_1-10_1_5,1.0,500.0,2,507,True,True,10.734,104,104,104,9.127,0.201,1.406
...,...,...,...,...,...,...,...,...,...,...,...,...,...
200x25_1-10_1_1,1.0,5000.0,2,1979,True,True,215.016,75,75,75,134.813,0.235,79.953
200x25_1-10_1_2,1.0,5000.0,2,2372,True,True,403.203,74,74,74,135.157,0.328,267.703
200x25_1-10_1_3,1.0,5000.0,2,2229,True,True,453.140,74,74,74,135.904,0.346,316.890
200x25_1-10_1_4,1.0,5000.0,2,2239,True,True,395.828,74,74,74,135.920,0.439,259.469




####################################################################### 050x10_1-10_1_1 ########################################################################

Instance specifications: J = 50 jobs, M = 10 machines, prec = 2 precedence relationship(s), and Rmax = 3 resources.

Instance_time_limit: 500.0
Iter_time_limit: 250.0

Constructive phase: alpha = 0.0
Search phase: False

GRASP iteration No. 113 starting at 494.849 / 500.0 s        
Repair iteration starting at 0.234 / 250.0 s        
Best solution: Cmax: 225, feasibility: True (precedence compliance: True, and resource compliance True)


####################################################################### 050x10_1-10_1_2 ########################################################################

Instance specifications: J = 50 jobs, M = 10 machines, prec = 3 precedence relationship(s), and Rmax = 3 resources.

Instance_time_limit: 500.0
Iter_time_limit: 250.0

Constructive phase: alpha = 0.0
Search phase: False

GRASP itera

,Alpha,T_limit,T_factor,Makespan,Feasibility,Repair,Time2best,Sol2best,Total_sol,GRASP iter,Construct_time,Search_time,Repair_time
050x10_1-10_1_1,0.0,500.0,2,225,True,True,9.109,113,113,113,8.672,0.187,0.234
050x10_1-10_1_2,0.0,500.0,2,237,True,True,9.031,114,114,114,8.589,0.175,0.235
050x10_1-10_1_3,0.0,500.0,2,201,True,True,9.140,113,113,113,8.686,0.158,0.296
050x10_1-10_1_4,0.0,500.0,2,262,True,True,9.172,114,114,114,8.594,0.234,0.344
050x10_1-10_1_5,0.0,500.0,2,201,True,True,8.984,114,114,114,8.718,0.110,0.156
...,...,...,...,...,...,...,...,...,...,...,...,...,...
200x25_1-10_1_1,0.0,5000.0,2,623,True,True,144.953,73,73,73,137.875,0.437,6.625
200x25_1-10_1_2,0.0,5000.0,2,783,True,True,168.860,72,72,72,137.646,0.324,30.875
200x25_1-10_1_3,0.0,5000.0,2,645,True,True,159.391,72,72,72,138.112,0.263,21.016
200x25_1-10_1_4,0.0,5000.0,2,626,True,True,149.203,72,72,72,138.470,0.358,10.359




####################################################################### 050x10_1-10_1_1 ########################################################################

Instance specifications: J = 50 jobs, M = 10 machines, prec = 2 precedence relationship(s), and Rmax = 3 resources.

Instance_time_limit: 500.0
Iter_time_limit: 250.0

Constructive phase: alpha = 0.0
Search phase: True

GRASP iteration No. 4 starting at 348.733 / 500.0 s        
Repair iteration starting at 0.219 / 250.0 s        
Best solution: Cmax: 223, feasibility: True (precedence compliance: True, and resource compliance True)


####################################################################### 050x10_1-10_1_2 ########################################################################

Instance specifications: J = 50 jobs, M = 10 machines, prec = 3 precedence relationship(s), and Rmax = 3 resources.

Instance_time_limit: 500.0
Iter_time_limit: 250.0

Constructive phase: alpha = 0.0
Search phase: True

GRASP iteration